# Check GPU version.

In [1]:
!nvidia-smi

Fri Jun 26 09:09:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Mount google drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Install TensorFlow-1.14 GPU.

In [3]:
# Select TensorFlow-1.x version.
%tensorflow_version 1.x

# Uninstall previous TensorFlow version.
!pip uninstall tensorflow -y 1>/dev/null 2>/dev/null 
!pip uninstall tensorflow-gpu -y 1>/dev/null 2>/dev/null 

# Install TensorFlow-1.14.
!pip install --upgrade tensorflow==1.14.0 1>/dev/null 2>/dev/null 
!pip install --upgrade tensorflow-gpu==1.14.0 1>/dev/null 2>/dev/null 

TensorFlow 1.x selected.


# Restart the runtime.

# Set the root directory.

In [1]:
import os

root_dir = '/content/'
os.chdir(root_dir)

!ls -al

total 20
drwxr-xr-x 1 root root 4096 Jun 26 09:10 .
drwxr-xr-x 1 root root 4096 Jun 26 09:08 ..
drwxr-xr-x 1 root root 4096 Jun 19 16:15 .config
drwx------ 4 root root 4096 Jun 26 09:10 drive
drwxr-xr-x 1 root root 4096 Jun 17 16:18 sample_data


# Import TensorFlow-1.14.

In [2]:
try:
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras import backend as K

import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

import numpy as np
np.random.seed(7)

import matplotlib.pyplot as plot

print(tf.__version__)

TensorFlow 1.x selected.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

# Download Flickr8K dataset.

### Download dataset.

In [ ]:
!gdown --id 15IPp8p_b4BrLuOIWAmm1jknt0Ip-WZ-F
!ls -al

### Extract dataset.

In [ ]:
!tar -xzf Flickr8K.tar.gz
!ls -al
!ls -al Flickr8K

# Load raw descriptions.

In [3]:
Flickr8K_root_dir = 'Flickr8K'
dataset_images_dir = os.path.join(Flickr8K_root_dir, 'images') + '/'

In [4]:
image_features_root_dir = 'Inception-v3'

In [5]:
descriptions_filename = os.path.join(image_features_root_dir, 'token.txt')
train_dataset_filename = os.path.join(image_features_root_dir, 'train_images.txt')
test_dataset_filename = os.path.join(image_features_root_dir, 'test_images.txt')

In [6]:
processed_descriptions_filename = os.path.join(image_features_root_dir, 'descriptions.txt')
train_features_filename = os.path.join(image_features_root_dir, 'train_features.pkl')
test_features_filename = os.path.join(image_features_root_dir, 'test_features.pkl')

In [7]:
def load_document(descriptions_filename):
	text_file = open(descriptions_filename, 'r')
	text_data = text_file.read()
	text_file.close()
	return( text_data )

### Load raw descriptions.

In [14]:
raw_descriptions = load_document(descriptions_filename)
print(raw_descriptions[:300])

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the s


# Parse descriptions.

In [15]:
def parse_descriptions(raw_descriptions):
	mapping = dict()

	for line in raw_descriptions.split('\n'):

		tokens = line.split()
		if len(line) < 2:
			continue

		image_id, image_descriptions = tokens[0], tokens[1:]
		image_id = image_id.split('.')[0]

		image_descriptions = ' '.join(image_descriptions)

		if image_id not in mapping:
			mapping[image_id] = list()

		mapping[image_id].append(image_descriptions)
	
	return mapping

### Parse descriptions.

In [16]:
descriptions = parse_descriptions(raw_descriptions)
print('loaded - %d descriptions.' % len(descriptions))

loaded - 8092 descriptions.


### View descriptions keys.

In [17]:
list(descriptions.keys())[:5]

['1000268201_693b08cb0e',
 '1001773457_577c3a7d70',
 '1002674143_1b742ab4b8',
 '1003163366_44323f5815',
 '1007129816_e794419615']

### View sample descriptions.

In [18]:
descriptions['1000268201_693b08cb0e']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

In [19]:
descriptions['1001773457_577c3a7d70']

['A black dog and a spotted dog are fighting',
 'A black dog and a tri-colored dog playing with each other on the road .',
 'A black dog and a white dog with brown spots are staring at each other in the street .',
 'Two dogs of different breeds looking at each other on the road .',
 'Two dogs on pavement moving toward each other .']

# Clean descriptions.

In [21]:
import string

def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)

### Clean descriptions.

In [22]:
clean_descriptions(descriptions)

### View cleaned sample descriptions.

In [23]:
descriptions['1000268201_693b08cb0e']

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin']

In [24]:
descriptions['1001773457_577c3a7d70']

['black dog and spotted dog are fighting',
 'black dog and tricolored dog playing with each other on the road',
 'black dog and white dog with brown spots are staring at each other in the street',
 'two dogs of different breeds looking at each other on the road',
 'two dogs on pavement moving toward each other']

# Create vocabulary of words.

In [25]:
def create_vocabulary(descriptions):
	all_descriptions = set() 
	for key in descriptions.keys():
		[all_descriptions.update(current_descriptions.split()) for current_descriptions in descriptions[key]]
	return( all_descriptions )

### Create vocabulary of words.

In [26]:
vocabulary = create_vocabulary(descriptions)
print('vcabulary size -', len(vocabulary))

vcabulary size - 8763


# Save descriptions.

In [ ]:
def save_descriptions(descriptions, filename):
	lines = list()
	for key, description_list in descriptions.items():
		for description in description_list:
			lines.append(key + ' ' + description)
	
	text_data = '\n'.join(lines)
	text_file = open(filename, 'w')
	text_file.write(text_data)
	text_file.close()

### Save descriptions.

In [ ]:
save_descriptions(descriptions, processed_descriptions_filename)

In [ ]:
!ls -al 'Inception-v3'

# Load dataset splits.

In [27]:
def load_dataset(filename):
	text_data = load_document(filename)
 
	dataset = list()
	for line in text_data.split('\n'):

		if len(line) < 1:
			continue

		identifier = line.split('.')[0]
		dataset.append(identifier)
	
	return set(dataset)

### Load training dataset split.

In [28]:
train_dataset = load_dataset(train_dataset_filename)
print('number of train dataset samples -', len(train_dataset))

number of train dataset samples - 6000


# Create a list of all image filenames in the directory.

In [ ]:
import glob

In [ ]:
image_filenames = glob.glob(dataset_images_dir + '*.jpg')

### View sample image filenames.

In [ ]:
print(image_filenames[:5])

### Read the train image filenames.

In [ ]:
train_image_filenames = set(open(train_dataset_filename, 'r').read().strip().split('\n'))

train_images = []
for image_filename in image_filenames: 
    if image_filename[len(dataset_images_dir):] in train_image_filenames: 
        train_images.append(image_filename)
print('number of training samples -',len(train_images))

### View sample train image filenames.

In [ ]:
print(train_images[:5])

### Read the test image filenames.

In [ ]:
test_image_filenames = set(open(test_dataset_filename, 'r').read().strip().split('\n'))

test_images = []
for image_filename in image_filenames: 
    if image_filename[len(dataset_images_dir):] in test_image_filenames: 
        test_images.append(image_filename)
print('number of test samples -',len(test_images))

### View sample test image filenames.

In [ ]:
print(test_images[:5])

# Load cleaned descriptions.

In [29]:
def load_cleaned_descriptions(filename, dataset):
	text_data = load_document(filename)
 
	descriptions = dict()
	for line in text_data.split('\n'):
		tokens = line.split()

		image_id, image_description = tokens[0], tokens[1:]

		if image_id in dataset:
			if image_id not in descriptions:
				descriptions[image_id] = list()

			current_description = 'startseq ' + ' '.join(image_description) + ' endseq'
			descriptions[image_id].append(current_description)
	 
	return( descriptions )

### Load cleaned decsriptions for train dataset split.

In [30]:
train_descriptions = load_cleaned_descriptions(processed_descriptions_filename, train_dataset)
print('number of training descriptions -' , len(train_descriptions))

number of training descriptions - 6000


# Preprocess an input image.

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image

In [ ]:
def preprocess(image_path):
    input_image = image.load_img(image_path, target_size=(299, 299))    
    input_image = image.img_to_array(input_image)

    input_image = np.expand_dims(input_image, axis=0)
    input_image = preprocess_input(input_image)
    return( input_image)

# Create feature extractor model.

### Load Inception-v3 model.

In [ ]:
base_model = InceptionV3(weights='imagenet')

### Create feature extractor model.

In [ ]:
feature_extractor = models.Model(base_model.input, base_model.layers[-2].output)

# Encode input images.

### Create function to encode input images.

In [ ]:
def encode_image(image_filename):
    input_image = preprocess(image_filename)
    image_features = feature_extractor.predict(input_image)
    image_features = np.reshape(image_features, image_features.shape[1])
    return( image_features )

### Create function to encode dataset.

In [ ]:
from time import time

def encode_images(input_images):
  start = time()
  feature_dictionary = {}
  for input_image in input_images:
    #print(input_image)
    feature_dictionary[input_image[len(dataset_images_dir):]] = encode_image(input_image)

  print("time taken in seconds -", ( time()-start ))

  return(feature_dictionary)

### Encode train dataset.

In [31]:
import pickle

### Encode train images and store features in a file.

In [32]:
train_features = encode_images(train_images)
with open(train_features_filename, 'wb') as pickle_file:
    pickle.dump(train_features, pickle_file)

NameError: ignored

### Encode test images and store features in a file.

In [ ]:
test_features = encode_images(test_images)
with open(test_features_filename, 'wb') as pickle_file:
    pickle.dump(test_features, pickle_file)

In [ ]:
!ls -al Inception-v3

In [ ]:
#!tar -czf Inception-v3.tar.gz Inception-v3
#!mv Inception-v3.tar.gz '/content/drive/My Drive/datasets/Flickr8K/.'

# OR

### Download image descriptions and Inception-v3 encoded image features from google drive.

In [8]:
!gdown --id 12f8iomzzeZNh0OVuLhjJkimKja21bYnn
!ls -al

Downloading...
From: https://drive.google.com/uc?id=12f8iomzzeZNh0OVuLhjJkimKja21bYnn
To: /content/Inception-v3.tar.gz
53.9MB [00:01, 53.1MB/s]
total 52684
drwxr-xr-x 1 root root     4096 Jun 26 09:13 .
drwxr-xr-x 1 root root     4096 Jun 26 09:08 ..
drwxr-xr-x 1 root root     4096 Jun 19 16:15 .config
drwx------ 4 root root     4096 Jun 26 09:10 drive
-rw-r--r-- 1 root root 53925692 Jun 26 09:13 Inception-v3.tar.gz
drwxr-xr-x 1 root root     4096 Jun 17 16:18 sample_data


In [9]:
!tar -xzf Inception-v3.tar.gz

In [10]:
!ls -al
!ls -al Inception-v3

total 52688
drwxr-xr-x 1 root root     4096 Jun 26 09:14 .
drwxr-xr-x 1 root root     4096 Jun 26 09:08 ..
drwxr-xr-x 1 root root     4096 Jun 19 16:15 .config
drwx------ 4 root root     4096 Jun 26 09:10 drive
drwxr-xr-x 2 root root     4096 Jun 26 08:12 Inception-v3
-rw-r--r-- 1 root root 53925692 Jun 26 09:13 Inception-v3.tar.gz
drwxr-xr-x 1 root root     4096 Jun 17 16:18 sample_data
total 63020
drwxr-xr-x 2 root root     4096 Jun 26 08:12 .
drwxr-xr-x 1 root root     4096 Jun 26 09:14 ..
-rw-r--r-- 1 root root  2943284 Jun 24 09:14 descriptions.txt
-rw-r--r-- 1 root root     1821 Oct 14  2013 readme.txt
-rw-r--r-- 1 root root  8284163 Jun 24 10:06 test_features.pkl
-rw-r--r-- 1 root root    25775 Oct 10  2013 test_images.txt
-rw-r--r-- 1 root root  3395237 Oct 14  2013 token.txt
-rw-r--r-- 1 root root 49708076 Jun 24 10:02 train_features.pkl
-rw-r--r-- 1 root root   154678 Oct 10  2013 train_images.txt


In [11]:
!rm Inception-v3.tar.gz
!ls -al

total 24
drwxr-xr-x 1 root root 4096 Jun 26 09:14 .
drwxr-xr-x 1 root root 4096 Jun 26 09:08 ..
drwxr-xr-x 1 root root 4096 Jun 19 16:15 .config
drwx------ 4 root root 4096 Jun 26 09:10 drive
drwxr-xr-x 2 root root 4096 Jun 26 08:12 Inception-v3
drwxr-xr-x 1 root root 4096 Jun 17 16:18 sample_data


# Load training dataset.

### Load training image features.

In [33]:
train_features = pickle.load(open(train_features_filename, 'rb'))
print('number of training samples -', len(train_features))

number of training samples - 6000


### Load training image descriptions.

# Create a list of all the training captions.

In [34]:
all_training_words = []
for key, current_descriptions in train_descriptions.items():
    for description in current_descriptions:
        all_training_words.append(description)

print('number of training words -', len(all_training_words))

number of training words - 30000


# Keep only words which occur at least a given number of times in the corpus.

In [35]:
word_count_threshold = 10

In [36]:
word_counts = {}
nsents = 0
for sent in all_training_words:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 7578 -> 1651


In [37]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [38]:
vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

1652

In [39]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('maximum description length -', max_length)

maximum description length - 34


In [40]:
# data generator, intended to be used in a call to model.fit_generator()
from numpy import array
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

Using TensorFlow backend.


# Load GloVe vectors.

In [12]:
!gdown --id 1-0SXOvbpDNvW8v3eFhKRqXb68xUQ86M1 # glove.6B.200d.txt

Downloading...
From: https://drive.google.com/uc?id=1-0SXOvbpDNvW8v3eFhKRqXb68xUQ86M1
To: /content/glove.6B.200d.txt
693MB [00:06, 107MB/s]


In [ ]:
!wget https://www.floydhub.com/api/v1/resources/Av2ThePYtAHXMAuSXEBV8X/glove.6B.200d.txt?content=true&rename=glove6b200dtxt
!mv 'glove.6B.200d.txt?content=true' glove.6B.200d.txt

In [13]:
!ls -al

total 677212
drwxr-xr-x 1 root root      4096 Jun 26 09:14 .
drwxr-xr-x 1 root root      4096 Jun 26 09:08 ..
drwxr-xr-x 1 root root      4096 Jun 19 16:15 .config
drwx------ 4 root root      4096 Jun 26 09:10 drive
-rw-r--r-- 1 root root 693432828 Jun 26 09:14 glove.6B.200d.txt
drwxr-xr-x 2 root root      4096 Jun 26 08:12 Inception-v3
drwxr-xr-x 1 root root      4096 Jun 17 16:18 sample_data


In [41]:
glove_dir = '/content/'
embedding_dim = 200

In [42]:
embeddings_index = {} 
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors.' % len(embeddings_index))

found 400000 word vectors.


In [43]:
# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [44]:
embedding_matrix.shape

(1652, 200)

# Create the model.

In [45]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.preprocessing.text import Tokenizer

In [46]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [47]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 200)      330400      input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
____________________________________________________________________________________________

In [48]:
model.layers[2]

In [49]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [50]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [51]:
epochs = 10
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)    
model.save_weights('model_10.h5')

In [ ]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)    
model.save_weights('model_20.h5')

Epoch 1/1
2000/2000 [==============================] - 195s 98ms/step - loss: 2.6838
Epoch 1/1
2000/2000 [==============================] - 194s 97ms/step - loss: 2.6580
Epoch 1/1
2000/2000 [==============================] - 195s 97ms/step - loss: 2.6312
Epoch 1/1
2000/2000 [==============================] - 196s 98ms/step - loss: 2.6125
Epoch 1/1
2000/2000 [==============================] - 196s 98ms/step - loss: 2.5909
Epoch 1/1
2000/2000 [==============================] - 197s 98ms/step - loss: 2.5750
Epoch 1/1
 496/2000 [======>.......................] - ETA: 2:27 - loss: 2.5432

In [ ]:
model.optimizer.lr = 0.0001
epochs = 10
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)    
model.save_weights('model.h5')

In [ ]:
model.load_weights('model.h5')

In [ ]:
images = '../../storage/image_caption/dataset/Flicker8k_Dataset/'

In [ ]:
with open("../../storage/image_caption/dataset/Pickle/encoded_test_images.pkl", "rb") as encoded_pickle:
    encoding_test = load(encoded_pickle)

In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
z+=1
pic = list(encoding_test.keys())[z]
image = encoding_test[pic].reshape((1,2048))
x=plt.imread(images+pic)
plt.imshow(x)
plt.show()
print("Greedy:",greedySearch(image))